In [1]:
import pandas as pd
from sqlalchemy import text
from connection import connect

In [2]:
co_oltp, etl_conn, _ = connect()

# Extract

In [3]:
query_reseller_sales = text("""
-- CTE para mapear sales_person_id (business_entity_id) → national_idnumber
WITH EmployeeMapping AS (
    SELECT
        e.business_entity_id,
        e.national_idnumber AS employee_national_id
    FROM hr.employee AS e
),
-- CTE para obtener el account_number por store_id
ResellerAccountMapping AS (
    SELECT
        c.store_id,
        MIN(c.account_number) AS account_number
    FROM sales.customer AS c
    WHERE c.store_id IS NOT NULL
    GROUP BY c.store_id
)
SELECT
    sod.sales_order_id,
    soh.sales_order_number,
    sod.sales_order_detail_id,
    ROW_NUMBER() OVER (PARTITION BY soh.sales_order_number ORDER BY sod.sales_order_detail_id) AS sales_order_line_number,
    sod.order_qty as order_quantity,
    sod.unit_price,
    (sod.unit_price * sod.order_qty) AS extended_amount,
    sod.unit_price_discount as unit_price_discount_pct,
    (sod.unit_price * sod.unit_price_discount * sod.order_qty) AS discount_amount,
    p.product_number,
    p.standard_cost AS product_standard_cost,
    (sod.order_qty * p.standard_cost) AS total_product_cost,
    (sod.unit_price * (1 - sod.unit_price_discount) * sod.order_qty) AS sales_amount,
    soh.tax_amt,
    soh.freight,
    soh.order_date,
    soh.due_date,
    soh.ship_date,
    soh.territory_id,
    soh.sales_person_id,
    em.employee_national_id,  -- ← El national_idnumber para hacer el merge
    soh.currency_rate_id,
    soh.purchase_order_number AS customer_po_number,
    soh.revision_number,
    sod.carrier_tracking_number,
    r.business_entity_id AS reseller_id,
    ram.account_number AS reseller_account_number  -- ← El account_number para hacer el merge
FROM sales.sales_order_header AS soh
JOIN sales.sales_order_detail AS sod
    ON soh.sales_order_id = sod.sales_order_id
JOIN production.product AS p
    ON sod.product_id = p.product_id
JOIN sales.customer AS c
    ON soh.customer_id = c.customer_id
JOIN sales.store AS r
    ON c.store_id = r.business_entity_id
LEFT JOIN ResellerAccountMapping AS ram
    ON r.business_entity_id = ram.store_id
LEFT JOIN EmployeeMapping AS em
    ON soh.sales_person_id = em.business_entity_id
WHERE
    soh.online_order_flag = false
    AND soh.sales_person_id IS NOT NULL;
""")

In [4]:
fact_reseller_sales = pd.read_sql(query_reseller_sales, co_oltp)
fact_reseller_sales

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number,...,ship_date,territory_id,sales_person_id,employee_national_id,currency_rate_id,customer_po_number,revision_number,carrier_tracking_number,reseller_id,reseller_account_number
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,2011-06-07 00:00:00+00:00,5,279,716374314,NaN,PO522145787,8,4911-403C-98,1046,AW00000676
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,2011-06-07 00:00:00+00:00,5,279,716374314,NaN,PO522145787,8,4911-403C-98,1046,AW00000676
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,2011-06-07 00:00:00+00:00,5,279,716374314,NaN,PO522145787,8,4911-403C-98,1046,AW00000676
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,2011-06-07 00:00:00+00:00,5,279,716374314,NaN,PO522145787,8,4911-403C-98,1046,AW00000676
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,2011-06-07 00:00:00+00:00,5,279,716374314,NaN,PO522145787,8,4911-403C-98,1046,AW00000676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60914,71952,SO71952,113559,37,2,158.430,316.860,0.0,0.0000,FR-M21S-52,...,2014-05-08 00:00:00+00:00,4,275,841560125,NaN,PO9715163911,8,9490-4552-81,1836,AW00000490
60915,71952,SO71952,113560,38,1,809.760,809.760,0.0,0.0000,FR-M94B-42,...,2014-05-08 00:00:00+00:00,4,275,841560125,NaN,PO9715163911,8,9490-4552-81,1836,AW00000490
60916,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,2014-05-08 00:00:00+00:00,4,275,841560125,NaN,PO9715163911,8,9490-4552-81,1836,AW00000490
60917,71952,SO71952,113562,40,3,32.394,97.182,0.0,0.0000,BB-7421,...,2014-05-08 00:00:00+00:00,4,275,841560125,NaN,PO9715163911,8,9490-4552-81,1836,AW00000490


# Transform

In [5]:
dim_product = pd.read_sql_table('dim_product', etl_conn)
# Contar cuántas veces aparece cada product_alternate_key
duplicate_products = dim_product['product_alternate_key'].value_counts()

# Mostrar las claves que tienen más de una entrada
print(duplicate_products[duplicate_products > 1])

product_alternate_key
HL-U509-R     3
HL-U509       3
FR-R38B-62    3
CA-1098       3
HL-U509-B     3
             ..
FR-R38R-58    2
HB-R720       2
HB-M918       2
HB-M763       2
HB-M243       2
Name: count, Length: 77, dtype: int64


In [6]:
dim_product['start_date'] = pd.to_datetime(dim_product['start_date'])
dim_product['end_date'] = pd.to_datetime(dim_product['end_date'])
dim_product['start_date'] = dim_product['start_date'].dt.tz_localize('UTC')
dim_product['end_date'] = dim_product['end_date'].dt.tz_localize('UTC')


In [7]:
merged_df = pd.merge(
    fact_reseller_sales,
    dim_product,
    left_on='product_number',
    right_on='product_alternate_key',
    how='left'
)

In [8]:
merged_df

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number,...,chinese_description,arabic_description,hebrew_description,thai_description,german_description,japanese_description,turkish_description,start_date,end_date,status
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98234,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,每个车架都是在我们位于 Bothell 的工厂中手工制作，具有高质量山地车架所必需的最佳直径...,وقد تم تصنيع كل هيكل من تلك الهياكل يدويًا في ...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, ל...",เฟรมแต่ละชิ้นผ่านการสร้างด้วยมือในโรงงาน Bothe...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
98235,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,每个车架都是在我们位于 Bothell 的工厂中手工制作，具有高质量山地车架所必需的最佳直径...,وقد تم تصنيع كل هيكل من تلك الهياكل يدويًا في ...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, ל...",เฟรมแต่ละชิ้นผ่านการสร้างด้วยมือในโรงงาน Bothe...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2012-05-30 00:00:00+00:00,2013-05-29 00:00:00+00:00,None
98236,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,每个车架都是在我们位于 Bothell 的工厂中手工制作，具有高质量山地车架所必需的最佳直径...,وقد تم تصنيع كل هيكل من تلك الهياكل يدويًا في ...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, 

In [9]:
# Condición para las filas con fecha de fin (registros históricos)
condicion_historica = (merged_df['order_date'] >= merged_df['start_date']) & (merged_df['order_date'] <= merged_df['end_date'])



In [10]:
condicion_actual = (merged_df['order_date'] >= merged_df['start_date']) & (merged_df['end_date'].isnull())


In [11]:
final_df = merged_df[condicion_historica | condicion_actual].copy()


In [12]:
print("Filas antes del merge:", len(fact_reseller_sales))
print("Filas después del merge y antes de filtrar:", len(merged_df))
print("Filas después de filtrar por fecha:", len(final_df))



Filas antes del merge: 60919
Filas después del merge y antes de filtrar: 98239
Filas después de filtrar por fecha: 60855


In [13]:
fact_reseller_sales = final_df

In [14]:
fact_reseller_sales

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number,...,chinese_description,arabic_description,hebrew_description,thai_description,german_description,japanese_description,turkish_description,start_date,end_date,status
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98230,71952,SO71952,113559,37,2,158.430,316.860,0.0,0.0000,FR-M21S-52,...,使用与 ML 铝制车架相同的开创性车架技术，经济实惠。,إنها تأتي ضمن أفضل منتجاتنا فنيًا حيث تستخدم ن...,האיכות הטובה ביותר שלנו שמנצלת טכנולוגיית מסגר...,ความคุ้มค่าที่มาพร้อมกับเทคโนโลยีการออกแบบเฟรม...,"Unser bester Wert ist die gleiche, bahnbrechen...","わたし たち の 最も 良 い 地 , すなわち 水晶 の ピソコ と し て の 地 を ...","ML Aluminum ähmily модельді serişdesi olaraq, ...",2013-05-30 00:00:00+00:00,NaT,Current
98233,71952,SO71952,113560,38,1,809.760,809.760,0.0,0.0000,FR-M94B-42,...,每个车架都是在我们位于 Bothell 的工厂中手工制作，具有高质量山地车架所必需的最佳直径...,وقد تم تصنيع كل هيكل من تلك الهياكل يدويًا في ...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, ל...",เฟรมแต่ละชิ้นผ่านการสร้างด้วยมือในโรงงาน Bothe...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current
98236,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,每个车架都是在我们位于 Bothell 的工厂中手工制作，具有高质量山地车架所必需的最佳直径...,وقد تم تصنيع كل هيكل من تلك الهياكل يدويًا في ...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, ל...",เฟรมแต่ละชิ้นผ่านการสร้างด้วยมือในโรงงาน Bothe...,Jede

In [18]:
## b. DimReseller
dim_reseller = pd.read_sql_table('dim_reseller', etl_conn)

# Asegurar tipos compatibles
fact_reseller_sales['reseller_account_number'] = fact_reseller_sales['reseller_account_number'].astype(str)
dim_reseller['reseller_alternate_key'] = dim_reseller['reseller_alternate_key'].astype(str)

fact_reseller_sales = fact_reseller_sales.merge(
    dim_reseller[['reseller_key', 'reseller_alternate_key']],
    left_on='reseller_account_number',
    right_on='reseller_alternate_key',
    how='left'
).drop(['reseller_id', 'reseller_account_number', 'reseller_alternate_key'], axis=1)

In [19]:
fact_reseller_sales

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number,...,arabic_description,hebrew_description,thai_description,german_description,japanese_description,turkish_description,start_date,end_date,status,reseller_key
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,71952,SO71952,113559,37,2,158.430,316.860,0.0,0.0000,FR-M21S-52,...,إنها تأتي ضمن أفضل منتجاتنا فنيًا حيث تستخدم ن...,האיכות הטובה ביותר שלנו שמנצלת טכנולוגיית מסגר...,ความคุ้มค่าที่มาพร้อมกับเทคโนโลยีการออกแบบเฟรม...,"Unser bester Wert ist die gleiche, bahnbrechen...","わたし たち の 最も 良 い 地 , すなわち 水晶 の ピソコ と し て の 地 を ...","ML Aluminum ähmily модельді serişdesi olaraq, ...",2013-05-30 00:00:00+00:00,NaT,Current,490
60851,71952,SO71952,113560,38,1,809.760,809.760,0.0,0.0000,FR-M94B-42,...,وقد تم تصنيع كل هيكل من تلك الهياكل يدويًا في ...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, ל...",เฟรมแต่ละชิ้นผ่านการสร้างด้วยมือในโรงงาน Bothe...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490
60852,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,وقد تم تصنيع كل هيكل من تلك الهياكل يدويًا في ...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, ל...",เฟรมแต่ละชิ้นผ่านการสร้างด้วยมือในโรงงาน Bothe...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490
60853,71952,SO71952,113562,40,3,32.394,97.182,0.0,0.0000,BB-7421,...,الفولاذ الكرومولي,פלדת כרומולי (כרום-מולובדניום),โลหะโครโมลี,Chromstahl.,"アビム , オフタ ,",Chromoil de

In [20]:
## c. DimEmployee (vendedor asignado)
dim_employee = pd.read_sql_table('dim_employee', etl_conn)

# Asegurar tipos compatibles
fact_reseller_sales['employee_national_id'] = fact_reseller_sales['employee_national_id'].astype(str)
dim_employee['employee_national_id_alternate_key'] = dim_employee['employee_national_id_alternate_key'].astype(str)

fact_reseller_sales = fact_reseller_sales.merge(
    dim_employee[['employee_key', 'employee_national_id_alternate_key']],
    left_on='employee_national_id',
    right_on='employee_national_id_alternate_key',
    how='left'
).drop(['sales_person_id', 'employee_national_id', 'employee_national_id_alternate_key'], axis=1)

In [21]:
fact_reseller_sales

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number,...,hebrew_description,thai_description,german_description,japanese_description,turkish_description,start_date,end_date,status,reseller_key,employee_key
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,אופני הרים תחרותיים מהשורה הראשונה. אפשרויות ל...,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,71952,SO71952,113559,37,2,158.430,316.860,0.0,0.0000,FR-M21S-52,...,האיכות הטובה ביותר שלנו שמנצלת טכנולוגיית מסגר...,ความคุ้มค่าที่มาพร้อมกับเทคโนโลยีการออกแบบเฟรม...,"Unser bester Wert ist die gleiche, bahnbrechen...","わたし たち の 最も 良 い 地 , すなわち 水晶 の ピソコ と し て の 地 を ...","ML Aluminum ähmily модельді serişdesi olaraq, ...",2013-05-30 00:00:00+00:00,NaT,Current,490,281
60851,71952,SO71952,113560,38,1,809.760,809.760,0.0,0.0000,FR-M94B-42,...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, ל...",เฟรมแต่ละชิ้นผ่านการสร้างด้วยมือในโรงงาน Bothe...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281
60852,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,"כל מסגרת מיוצרת ידנית במפעל שלנו ב- Bothell, ל...",เฟรมแต่ละชิ้นผ่านการสร้างด้วยมือในโรงงาน Bothe...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281
60853,71952,SO71952,113562,40,3,32.394,97.182,0.0,0.0000,BB-7421,...,פלדת כרומולי (כרום-מולובדניום),โลหะโครโมลี,Chromstahl.,"アビム , オフタ ,",Chromoil demir.,2013-05-30 00:00:00+00:00,NaT,Current,490,281


In [22]:
t_special_offer = pd.read_sql("""
    SELECT p.product_number, sod.special_offer_id
    FROM sales.sales_order_detail AS sod
    JOIN production.product AS p ON sod.product_id = p.product_id
""", co_oltp)

dim_promotion = pd.read_sql_table('dim_promotion', etl_conn)
fact_reseller_sales = fact_reseller_sales.merge(
    t_special_offer,
    left_on='sales_order_detail_id',
    right_index=True,
    how='left'
)

In [23]:
fact_reseller_sales

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number_x,...,german_description,japanese_description,turkish_description,start_date,end_date,status,reseller_key,employee_key,product_number_y,special_offer_id
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82B-44,1
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82B-48,1
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-38,1
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-42,1
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,71952,SO71952,113559,37,2,158.430,316.860,0.0,0.0000,FR-M21S-52,...,"Unser bester Wert ist die gleiche, bahnbrechen...","わたし たち の 最も 良 い 地 , すなわち 水晶 の ピソコ と し て の 地 を ...","ML Aluminum ähmily модельді serişdesi olaraq, ...",2013-05-30 00:00:00+00:00,NaT,Current,490,281,FR-M94B-42,1
60851,71952,SO71952,113560,38,1,809.760,809.760,0.0,0.0000,FR-M94B-42,...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281,FR-M94S-46,1
60852,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281,BB-7421,1
60853,71952,SO71952,113562,40,3,32.394,97.182,0.0,0.0000,BB-7421,...,Chromstahl.,"アビム , オフタ ,",Chromoil demir.,2013-05-30 00:00:00+00:00,NaT,Current,490,281,BK-M18S-42,16


In [24]:
fact_reseller_sales = fact_reseller_sales.merge(
    dim_promotion[['promotion_key', 'promotion_alternate_key']],
    left_on='special_offer_id',
    right_on='promotion_alternate_key',
    how='left'
).drop(['special_offer_id', 'promotion_alternate_key'], axis=1)

fact_reseller_sales

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number_x,...,german_description,japanese_description,turkish_description,start_date,end_date,status,reseller_key,employee_key,product_number_y,promotion_key
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82B-44,1
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82B-48,1
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-38,1
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-42,1
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,Top-of-the-line-Wettbewerb Mountainbike. Leist...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,71952,SO71952,113559,37,2,158.430,316.860,0.0,0.0000,FR-M21S-52,...,"Unser bester Wert ist die gleiche, bahnbrechen...","わたし たち の 最も 良 い 地 , すなわち 水晶 の ピソコ と し て の 地 を ...","ML Aluminum ähmily модельді serişdesi olaraq, ...",2013-05-30 00:00:00+00:00,NaT,Current,490,281,FR-M94B-42,1
60851,71952,SO71952,113560,38,1,809.760,809.760,0.0,0.0000,FR-M94B-42,...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281,FR-M94S-46,1
60852,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,Jeder Rahmen wird in unserer Bothell-Anlage in...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281,BB-7421,1
60853,71952,SO71952,113562,40,3,32.394,97.182,0.0,0.0000,BB-7421,...,Chromstahl.,"アビム , オフタ ,",Chromoil demir.,2013-05-30 00:00:00+00:00,NaT,Current,490,281,BK-M18S-42,16


In [25]:
## e. DimCurrency
dim_currency = pd.read_sql_table('dim_currency', etl_conn)
t_currency = pd.read_sql("SELECT currency_rate_id, to_currency_code FROM sales.currency_rate", co_oltp)

# Añadir el código de la moneda
fact_reseller_sales = fact_reseller_sales.merge(
    t_currency,
    on='currency_rate_id',
    how='left'
)

# Para los NULLs en currency_rate_id, usar una moneda por defecto
default_currency_code = 'USD'

fact_reseller_sales['to_currency_code'] = fact_reseller_sales['to_currency_code'].fillna(default_currency_code)

fact_reseller_sales.drop('currency_rate_id', axis=1, inplace=True)

# Reemplazar el código de la moneda por la llave subrogada
fact_reseller_sales = fact_reseller_sales.merge(
    dim_currency[['currency_key', 'currency_alternate_key']],
    left_on='to_currency_code',
    right_on='currency_alternate_key',
    how='left'
)

# Eliminar las columnas de unión restantes
fact_reseller_sales.drop(['to_currency_code', 'currency_alternate_key'], axis=1, inplace=True)

fact_reseller_sales

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number_x,...,japanese_description,turkish_description,start_date,end_date,status,reseller_key,employee_key,product_number_y,promotion_key,currency_key
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82B-44,1,100
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82B-48,1,100
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-38,1,100
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-42,1,100
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-44,1,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,71952,SO71952,113559,37,2,158.430,316.860,0.0,0.0000,FR-M21S-52,...,"わたし たち の 最も 良 い 地 , すなわち 水晶 の ピソコ と し て の 地 を ...","ML Aluminum ähmily модельді serişdesi olaraq, ...",2013-05-30 00:00:00+00:00,NaT,Current,490,281,FR-M94B-42,1,100
60851,71952,SO71952,113560,38,1,809.760,809.760,0.0,0.0000,FR-M94B-42,...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281,FR-M94S-46,1,100
60852,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281,BB-7421,1,100
60853,71952,SO71952,113562,40,3,32.394,97.182,0.0,0.0000,BB-7421,...,"アビム , オフタ ,",Chromoil demir.,2013-05-30 00:00:00+00:00,NaT,Current,490,281,BK-M18S-42,16,100


In [26]:
fact_reseller_sales

,sales_order_id,sales_order_number,sales_order_detail_id,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_number_x,...,japanese_description,turkish_description,start_date,end_date,status,reseller_key,employee_key,product_number_y,promotion_key,currency_key
0,43659,SO43659,1,1,1,2024.994,2024.994,0.0,0.0000,BK-M82B-42,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82B-44,1,100
1,43659,SO43659,2,2,3,2024.994,6074.982,0.0,0.0000,BK-M82B-44,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82B-48,1,100
2,43659,SO43659,3,3,1,2024.994,2024.994,0.0,0.0000,BK-M82B-48,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-38,1,100
3,43659,SO43659,4,4,1,2039.994,2039.994,0.0,0.0000,BK-M82S-38,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-42,1,100
4,43659,SO43659,5,5,1,2039.994,2039.994,0.0,0.0000,BK-M82S-42,...,"焼きカシャ 山 を 打 た れ る 者 に は ロクミ 山 に あ り , クミン に は ...",Parça-furl-dağ ярыш bisiklopedina.Phone festiv...,2011-05-31 00:00:00+00:00,2012-05-29 00:00:00+00:00,None,676,285,BK-M82S-44,1,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,71952,SO71952,113559,37,2,158.430,316.860,0.0,0.0000,FR-M21S-52,...,"わたし たち の 最も 良 い 地 , すなわち 水晶 の ピソコ と し て の 地 を ...","ML Aluminum ähmily модельді serişdesi olaraq, ...",2013-05-30 00:00:00+00:00,NaT,Current,490,281,FR-M94B-42,1,100
60851,71952,SO71952,113560,38,1,809.760,809.760,0.0,0.0000,FR-M94B-42,...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281,FR-M94S-46,1,100
60852,71952,SO71952,113561,39,4,818.700,3274.800,0.0,0.0000,FR-M94S-46,...,"われわれ の 手 に は か れ て い る . われわれ の 手 の 上 に は , クミ...",Һәр frank Countdown referens we diagramy Dağ к...,2013-05-30 00:00:00+00:00,NaT,Current,490,281,BB-7421,1,100
60853,71952,SO71952,113562,40,3,32.394,97.182,0.0,0.0000,BB-7421,...,"アビム , オフタ ,",Chromoil demir.,2013-05-30 00:00:00+00:00,NaT,Current,490,281,BK-M18S-42,16,100


In [27]:
## f. DimSalesTerritory
dim_sales_territory = pd.read_sql_table('dim_sales_territory', etl_conn)
fact_reseller_sales = fact_reseller_sales.merge(
    dim_sales_territory[['sales_territory_key', 'sales_territory_alternate_key']],
    left_on='territory_id',
    right_on='sales_territory_alternate_key',
    how='left'
).drop(['territory_id', 'sales_territory_alternate_key'], axis=1)

In [28]:
fact_reseller_sales['order_date_key'] = fact_reseller_sales['order_date'].dt.strftime('%Y%m%d').astype(int)
fact_reseller_sales['due_date_key'] = fact_reseller_sales['due_date'].dt.strftime('%Y%m%d').astype(int)
fact_reseller_sales['ship_date_key'] = fact_reseller_sales['ship_date'].dt.strftime('%Y%m%d').astype(int)

In [29]:
final_columns = [
    'product_key',
    'order_date_key',
    'due_date_key',
    'ship_date_key',
    'reseller_key',
    'employee_key',
    'promotion_key',
    'currency_key',
    'sales_territory_key',
    'sales_order_number',
    'sales_order_line_number',
    'revision_number',
    'order_quantity',
    'unit_price',
    'extended_amount',
    'unit_price_discount_pct',
    'discount_amount',
    'product_standard_cost',
    'total_product_cost',
    'sales_amount',
    'tax_amt',
    'freight',
    'carrier_tracking_number',
    'customer_po_number',
    'order_date',
    'due_date',
    'ship_date'
]

df_to_load = fact_reseller_sales[final_columns]

In [30]:
df_to_load

,product_key,order_date_key,due_date_key,ship_date_key,reseller_key,employee_key,promotion_key,currency_key,sales_territory_key,sales_order_number,...,product_standard_cost,total_product_cost,sales_amount,tax_amt,freight,carrier_tracking_number,customer_po_number,order_date,due_date,ship_date
0,349,20110531,20110612,20110607,676,285,1,100,5,SO43659,...,1898.0944,1898.0944,2024.9940,1971.5149,616.0984,4911-403C-98,PO522145787,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00
1,350,20110531,20110612,20110607,676,285,1,100,5,SO43659,...,1898.0944,5694.2832,6074.9820,1971.5149,616.0984,4911-403C-98,PO522145787,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00
2,351,20110531,20110612,20110607,676,285,1,100,5,SO43659,...,1898.0944,1898.0944,2024.9940,1971.5149,616.0984,4911-403C-98,PO522145787,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00
3,344,20110531,20110612,20110607,676,285,1,100,5,SO43659,...,1912.1544,1912.1544,2039.9940,1971.5149,616.0984,4911-403C-98,PO522145787,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00
4,345,20110531,20110612,20110607,676,285,1,100,5,SO43659,...,1912.1544,1912.1544,2039.9940,1971.5149,616.0984,4911-403C-98,PO522145787,2011-05-31 00:00:00+00:00,2011-06-12 00:00:00+00:00,2011-06-07 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,527,20140501,20140513,20140508,490,281,1,100,4,SO71952,...,144.5938,289.1876,316.8600,6573.0031,2054.0635,9490-4552-81,PO9715163911,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00
60851,298,20140501,20140513,20140508,490,281,1,100,4,SO71952,...,739.0410,739.0410,809.7600,6573.0031,2054.0635,9490-4552-81,PO9715163911,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00
60852,295,20140501,20140513,20140508,490,281,1,100,4,SO71952,...,747.2002,2988.8008,3274.8000,6573.0031,2054.0635,9490-4552-81,PO9715163911,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00
60853,601,20140501,20140513,20140508,490,281,16,100,4,SO71952,...,23.9716,71.9148,97.1820,6573.0031,2054.0635,9490-4552-81,PO9715163911,2014-05-01 00:00:00+00:00,2014-05-13 00:00:00+00:00,2014-05-08 00:00:00+00:00


# Load

In [31]:
df_to_load.to_sql(
    'fact_reseller_sales',
    etl_conn,
    schema='dw',
    if_exists='append',
    index=False
)

855